In [ ]:
# Passo 1: Importar as bibliotecas necessárias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import pickle
import numpy as np
import open3d as o3d

In [ ]:
# Definir os caminhos
data_folder = "nuvem_de_pontos/"
DATANAME = "3DML_urban_point_cloud.xyz"

# Carregar os dados
pcd = pd.read_csv(data_folder + DATANAME, delimiter=' ')
pcd.dropna(inplace=True)

In [ ]:
# Preparar os dados
labels = pcd['Classification']
features = pcd[['X', 'Y', 'Z', 'R', 'G', 'B']]
features_scaled = MinMaxScaler().fit_transform(features)

# Dividir os dados
X_train, X_test, y_train, y_test = train_test_split(features_scaled, labels, test_size=0.4)

# Treinar o classificador
rf_classifier = RandomForestClassifier(n_estimators=10)
rf_classifier.fit(X_train, y_train)

In [ ]:
# Avaliar o modelo
rf_predictions = rf_classifier.predict(X_test)
print(classification_report(y_test, rf_predictions, target_names=['ground', 'vegetation', 'buildings']))

# Salvar o modelo treinado
model_filename = data_folder + "treinado\\aprendizado.pkl"
with open(model_filename, 'wb') as model_file:
    pickle.dump(rf_classifier, model_file)

print(f"Modelo salvo em: {model_filename}")

In [ ]:
# Definir o caminho para o modelo salvo e para os novos dados
path = "treinado\\aprendizado.pkl"
model_path = data_folder + path
data = "TESTE.xyz"
new_data_path = data_folder + data

# Carregar o modelo treinado
with open(model_path, 'rb') as model_file:
    rf_classifier = pickle.load(model_file)

# Carregar e preparar a nova nuvem de pontos
# Lê o arquivo ignorando a primeira linha (cabeçalho comentado)
new_pcd = pd.read_csv(new_data_path, delimiter=' ', header=None, skiprows=1)

# Define os nomes das colunas conforme o cabeçalho informado
columns = ['X', 'Y', 'Z', 'R', 'G', 'B', 'Intensity', 'Return_Number', 'Number_Of_Returns', 'Scan_Direction_Flag', 'Classification', 'Scan_Angle', 'User_Data', 'Point_Source_ID', 'Gps_Time', 'Near_Infrared']
new_pcd.columns = columns

# Selecionar somente as colunas relevantes para a classificação
new_features = new_pcd[['X', 'Y', 'Z', 'R', 'G', 'B']]
new_features_scaled = MinMaxScaler().fit_transform(new_features)

# Classificar a nova nuvem de pontos
new_predictions = rf_classifier.predict(new_features_scaled)

# Adicionar as previsões ao DataFrame e salvar os resultados
new_pcd['Classification'] = new_predictions
output_path = "classificado.xyz"
new_pcd.to_csv(data_folder + output_path, index=False, sep=' ')

print(f"Nuvem de pontos classificada salva em: {output_path}")

In [ ]:
# Vizualizar a Nuvem de Pontos
# Carregar os dados da nuvem de pontos
data = np.loadtxt(data_folder + DATANAME, skiprows=1, delimiter=" ")


# Extrair as coordenadas XYZ
points = data[:, 0:3]

# Extrair as cores RGB (opcional)
colors = data[:, 3:6] / 255.0

# Criar um ponto nuvem Open3D
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)

# Colocando cores originais
if colors is not None:
    pcd.colors = o3d.utility.Vector3dVector(colors)

# Visualizar a nuvem de pontos
o3d.visualization.draw_geometries([pcd])

In [ ]:
# Vizualizar a Nuvem de Pontos Classificada
# Extrair as classificações
classifications = data[:, 13]

# Cores para cada classe
colors = {
    "1.0": [0.1, 0.8, 0.1],  # Terreno (verde)
    "2.0": [0.0, 0.5, 0.0],  # Vegetação (verde escuro)
    "3.0": [0.8, 0.6, 0.2],  # Construção (marrom)
}

# Mapear as classificações para cores
point_colors = [colors[str(c)] for c in classifications]

# Criar um ponto nuvem Open3D
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = o3d.utility.Vector3dVector(point_colors)

# Visualizar a nuvem de pontos Classificada
o3d.visualization.draw_geometries([pcd])